In [1]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import*
from PIL import Image,ImageTk
import sys
interface=tk.Tk()
interface.title('Converting Image')
interface.minsize(1000, 600)
temp = Image.open('C:\\Users\\sbg\\Desktop\\S1AIDS\\back.png')
back = ImageTk.PhotoImage(temp)
backImg = Label(interface,image=back).place(x=0,y=0)

def open_image():
    global filename,imgtk,img
    filename = filedialog.askopenfilename(initialdir='/',title='Select an image please',filetypes=(('jpg files','*.jpg'),('png files','*.png')))
    img = Image.open(filename)
    resized = img.resize((320,320))
    imgtk = ImageTk.PhotoImage(resized)
    Label(interface,image=imgtk).place(x=340,y=80)
btn_select_image=Button(interface,text='Browse images',command=open_image,cursor='hand2',bg='#FDFEFE',font='Helvetica 10 bold')
btn_select_image.grid(column=0,row=1)
operation=Label(interface,text='Number of cluster : K ',font='Helvetica 7',bg='powderblue')
operationn=Label(interface,text='Number of iterations',font='Helvetica 7',bg='powderblue')
operation.place(x=800,y=100)
operationn.place(x=800,y=150)
Q = np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]])

def DCT(u,v,x,y):
    return np.cos(((2*x+1)*u*np.pi)/16)*np.cos(((2*y+1)*v*np.pi)/16)

def filter_bank(block_size):
    filter_bank = np.zeros((block_size,block_size,block_size,block_size))
    for u in range(filter_bank.shape[0]):
        for v in range(filter_bank.shape[1]):
            for x in range(filter_bank.shape[2]):
                for y in range(filter_bank.shape[3]):
                    filter_bank[u,v,x,y] = DCT(u,v,x,y)
    return filter_bank

def dct_coeff(block,filter_bank,quant=Q):
    if((block.shape[0]!=8) or block.shape[1]!=block.shape[0]):
        raise Exception('Only 8x8 blocks are allowed')
    else:
        block,G = block-128.,np.empty_like(block).astype('float16')
        a = 1/np.sqrt(2)
        for u in range(8):
            for v in range(8):
                if((u!=0) and (v!=0)):
                    G[u,v] = (1/4)*sum(sum(block*filter_bank[u,v]))
                elif(u==0 and v==0):
                    G[u,v] = (1/4)*(a**2)*sum(sum(block*filter_bank[u,v]))
                else:
                    G[u,v] = (1/4)*a*sum(sum(block*filter_bank[u,v])) 
    G = np.round(G/quant)
    G[G== -0.] = 0.
    return G

def inv_dct(block,filter_bank,quant=Q):
    if((block.shape[0]!=8) or block.shape[1]!=block.shape[0]):
        raise Exception('Only 8x8 blocks are allowed')
    else:
        G,a= np.empty_like(block), 1/np.sqrt(2)
        for u in range(8):
            for v in range(8):
                if((u!=0) and (v!=0)):
                    G[u,v] = (1/4)*sum(sum(block*filter_bank[u,v]))
                elif(u==0 and v==0):
                    G[u,v] = (1/4)*sum(sum((a**2)*block*filter_bank[u,v]))
                else:
                    G[u,v] = (1/4)*sum(sum(a*block*filter_bank[u,v]))
    return np.round(G)*Q+128

f = filter_bank(8)

def run_len(vector):
    i, encoded, occ_char, vect_len , end = 0, np.array([]) , 0, len(vector), False
    while i < vect_len:
        if (vector[i] == 0):
            occ_char+=1
            if(i==vect_len-1):
                encoded = np.append(encoded,(0,occ_char))
        elif (vector[i] != 0):
            if occ_char>0:
                if occ_char == 1:
                    encoded = np.append(encoded,-1)
                else:
                    encoded = np.append(encoded,(0,occ_char))
                occ_char = 0
            encoded = np.append(encoded,vector[i])
        i+=1
    return encoded
        
def runlendec(vect):
    index,vector = 0,[]
    while index < len(vect):
        if(vect[index]==0 and vect[index+1]>0):
            vector.extend([0]*int(vect[index+1]))
            index+=2
        else:
            if(vect[index] == -1):
                vector.append(0)
            else:
                vector.append(vect[index])
            index+=1
    return np.array(vector)

def zigzag(block):
    if((block.shape[0]!=8) or block.shape[1]!=block.shape[0]):
        raise Exception('Only 8x8 blocks are allowed')
    else:
        vector = np.array([block[0,0]])
        for row in range(1,8,1):
            index, sub_img = list(np.arange(0,row+1)), block[:row+1,:row+1]
            if((row+1)%2==1):
                vector = np.append(vector,(sub_img[index,index[::-1]])[::-1])
            else:
                vector = np.append(vector,sub_img[index,index[::-1]])
        for row in range(1,8,1):
            index = list(np.arange(row,8))
            if((row+1)%2==0):
                vector = np.append(vector,block[index,index[::-1]][::-1])
            else:
                vector = np.append(vector,block[index,index[::-1]])
        return vector
    
def inv_zigzag(vector):
    if len(vector) != 64:
        raise Exception(str(len(vector)) + 'elements could not form an 8x8 block')
    block = np.empty((8,8))
    block[0,0],point= vector[0],1
    for row in range(1,8,1):
        index, sub_img = list(np.arange(0,row+1)), block[:row+1,:row+1]
        if((row+1)%2==1):
            sub_img[index,index[::-1]] = vector[point:point+(sub_img.shape[0])][::-1]
        else:
            sub_img[index,index[::-1]] = vector[point:point+(sub_img.shape[0])]
        point+=sub_img.shape[0]
    for row in range(1,8,1):
        index = list(np.arange(row,8))
        if((row+1)%2==0):
            block[index,index[::-1]] = vector[point:point+(block[index,index[::-1]].shape[0])][::-1]
        else:
            block[index,index[::-1]] = vector[point:point+( block[index,index[::-1]].shape[0])]
        point+= block[index,index[::-1]].shape[0]
    return block

def rgbtoycbcr(img):
    img,vals = np.array(img),np.array([[0.299 ,0.587, 0.114],[-0.1687,-0.3313,0.5] ,[0.5,-0.4187,-0.0813]])
    ycbcr = img@vals.T
    ycbcr[:,:,[1,2]]+=128
    return ycbcr

def zigzag_img(img):
    vect_img = []
    for row_block in range(0,img.shape[0],8):
        for col_block in range(0,img.shape[1],8):
            vect_img.append(zigzag(img[row_block:row_block+8,col_block:col_block+8]))
    return vect_img

def resize_rgb(img):
    r,g,b,band,shape,c = img[:,:,0],img[:,:,1],img[:,:,2],[],np.array([img.shape[0],img.shape[1]]),False
    if(img.shape[0]%8!=0):
        shape[0],c = shape[0] + (8-shape[0]%8), True
    if(img.shape[1]%8!=0):
        shape[1],c = shape[1] + (8-shape[1]%8), True
    if not c :
        return img
    else:
        shape = tuple(shape)
        r_ = g_ = b_ = np.full(shape,-1)
        r_[:img.shape[0],:img.shape[1]],g_[:img.shape[0],:img.shape[1]],b_[:img.shape[0],:img.shape[1]] = r,g,b
        for row_pix in range(img.shape[0]-1,shape[0],1):
            for col_pix in range(shape[1]):
                if(r_[row_pix-1,col_pix] != -1):
                    r_[row_pix,col_pix],g_[row_pix,col_pix],b_[row_pix,col_pix] =r_[row_pix-1,col_pix],g_[row_pix-1,col_pix],b_[row_pix-1,col_pix]
        for col_pix in range(img.shape[1]-1,shape[1],1):
            for row_pix in range(shape[0]):
                if(r_[row_pix,col_pix-1] != -1):
                     r_[row_pix,col_pix],g_[row_pix,col_pix],b_[row_pix,col_pix] =r_[row_pix,col_pix-1],g_[row_pix,col_pix-1],b_[row_pix,col_pix-1]
        r_,g_,b_ = r_.reshape((r_.shape[0],r_.shape[1],1)),g_.reshape((g_.shape[0],g_.shape[1],1)),b_.reshape((b_.shape[0],b_.shape[1],1))
        return Image.fromarray(np.concatenate((r_,g_,b_),axis=2).astype(np.uint8))
    
# def ycbcr2rgb(rgbimg):
#     vals = np.array([[1,0,1.402],[1,-0.34414,-0.71414],[1,1.772,0]])
#     rgbimg[:,:,[1,2]]-=128
#     rgbimg = rgbimg@vals.T
#     return rgbimg.astype('uint8')
  
def compress():
    global img,comp_cb,comp_cr,shape
    img = rgbtoycbcr(np.array(resize_rgb(np.array(Image.open(filename)))))
    shape = img.shape
    for row in range(0,shape[0],8):
        for col in range(0,shape[1],8):
            (img[:,:,1])[row:row+8,col:col+8],(img[:,:,2])[row:row+8,col:col+8] = dct_coeff((img[:,:,1])[row:row+8,col:col+8],f),dct_coeff((img[:,:,2])[row:row+8,col:col+8],f)
    vect_cb,vect_cr = zigzag_img(img[:,:,1]),zigzag_img(img[:,:,2])
    comp_cb,comp_cr = [run_len(vect) for vect in vect_cb],[run_len(vect) for vect in vect_cr] 
    
def reconstruct():
    global image,imagetk
    compress()
    image,bands,vect = np.zeros(shape),[comp_cb,comp_cr],0
    for band in range(len(bands)):
            for rowpix in range(0,image.shape[0],8):
                for colpix in range(0,image.shape[1],8):
                    image[rowpix:rowpix+8,colpix:colpix+8,band+1] = inv_dct(inv_zigzag(runlendec(bands[band][vect])),f)
                    vect+=1
            vect = 0
    image[:,:,0] = img[:,:,0]
    Image.fromarray(image.astype('uint8'),mode='YCbCr').show()
    print(sys.getsizeof(img),sys.getsizeof((np.array((comp_cb,comp_cr)))) + sys.getsizeof(img[:,:,0])) 
    
DCT=Button(interface,text='DCT(jpg)',cursor='hand2',bg='#FDFEFE',font='Helvetica 10 bold',command=reconstruct)
DCT.place(x=800,y=30)


# def getsize():
#     global text
#     text = 

global k,itr
k  = StringVar(interface)
itr = StringVar(interface)
entry =  Entry(interface,textvariable=k)
entryy =  Entry(interface,textvariable=itr)

def RGBtoGray(image):
    rowimage = np.asarray(image,dtype='float')
    if(image.mode=='RGB'):
        r,g,b = rowimage[:,:,0],rowimage[:,:,1],rowimage[:,:,2]
        return Image.fromarray((r*0.299+g*0.587+b*0.114))
    else:
        return Image.fromarray(rowimage)


class KMeans():
    def _init_centers(self, points, K):

            row, col = points.shape
            retArr = np.empty([K, col]) # contains the K cluster centers 
            for number in range(K):
                randIndex = np.random.randint(row)
                retArr[number] = points[randIndex]
            return retArr
        
    def dist(self, x, y):
        xSumSquare = np.sum(np.square(x),axis=1);
        ySumSquare = np.sum(np.square(y),axis=1);
        mul = np.dot(x, y.T);
        dists = np.sqrt(abs(xSumSquare[:, np.newaxis] + ySumSquare-2*mul))
        return dists
    
    def _update_assignment(self, centers, points):
        row, col = points.shape
        cluster_idx = np.empty([row])
        distances = self.dist(points, centers)
        cluster_idx = np.argmin(distances, axis=1)
        return cluster_idx
    
    def _update_centers(self, old_centers, cluster_idx, points):
        K, D = old_centers.shape
        new_centers = np.empty(old_centers.shape)
        for i in range(K):
            new_centers[i] = np.mean(points[cluster_idx == i], axis = 0)
        return new_centers
    
    def loss(self, centers, cluster_idx, points):
        dists = self.dist(points, centers)
        loss = 0.0
        N, D = points.shape
        for i in range(N):
            loss = loss + np.square(dists[i][cluster_idx[i]])
        return loss
    
    def call(self, points, K):
            max_iters=int(itr.get())
            centers = self._init_centers(points, K)
            for it in range(max_iters):
                cluster_idx = self._update_assignment(centers, points)
                centers = self._update_centers(centers, cluster_idx, points)
                loss = self.loss(centers, cluster_idx, points)
                K = centers.shape[0]
            return cluster_idx, centers, loss
def segment():
    global img
    K = KMeans()
    k = int(entry.get())
    dif = int(255/k)
    img,colors = np.array(RGBtoGray(Image.open(filename))),[i*dif for i in range(k)]
    shape = img.shape
    img = img.reshape((np.product(shape),1))
    clustered = K.call(img,k)[0]
    clustered = clustered.reshape(img.shape)
    for cluster in range(k):
        img[clustered==cluster] = colors[cluster]
    img = Image.fromarray(img.reshape(shape)).show()

kmeans=Button(interface,text='KMeans(segment)',cursor='hand2',bg='#FDFEFE',font='Helvetica 10 bold',command=segment)
kmeans.place(x=800,y=60)
entry.place(x=800,y=120)
entryy.place(x=800,y=170)
btn9=Button(interface,text='Exit Program',command=interface.quit,cursor='hand2',bg='skyblue')
btn9.place(x=800,y=500)

interface.mainloop()